In [20]:
import gdown

In [21]:
url = "https://drive.google.com/file/d/1Ri35otP5KdbnthLSfIAKRhyHUKKJjTSJ/view?usp=sharing"


In [22]:
file_id = url.split('/')[-2]


In [23]:
prefix = "https://drive.google.com/uc?/export=download&id="

In [24]:
gdown.download(prefix + file_id , "archive(6).zip")

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1Ri35otP5KdbnthLSfIAKRhyHUKKJjTSJ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1Ri35otP5KdbnthLSfIAKRhyHUKKJjTSJ&confirm=t&uuid=5a7ce23a-fe90-42eb-bbe2-5253bd67e48f
To: c:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\research\archive(6).zip
100%|██████████| 124M/124M [00:31<00:00, 4.00MB/s] 


'archive(6).zip'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

In [26]:
from chest_cancer_classifier.constants import *
from chest_cancer_classifier.utils.common import read_yaml,create_dir

In [27]:
class ConfigManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        
        create_dir([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_dir([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_root_dir = config.unzio_dir
        )
        
        return data_ingestion_config

In [28]:
import os 
import urllib.request as request
import zipfile
import gdown
from chest_cancer_classifier import logger
from chest_cancer_classifier.utils.common import get_size


In [31]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/dataingestion",exist_ok = True)
            logger.info(f"Downloading dat from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split('/')[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id , "archive(6).zip")
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_file:
            zip_file.extractall(unzip_path)

In [33]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'